In [ ]:
import timeit
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import gudhi as gd
import gudhi.representations as gdr
from alpha_complex_periodic import calc_persistence
from gadgetutils import utils

In [ ]:
plt.style.use(["science", "notebook"])

# Timing tests

In [ ]:
N_vals = np.logspace(1.9, 4.6, 12, dtype=int)

In [ ]:
def pers_reg(points):
    st = gd.AlphaComplex(points=points).create_simplex_tree()
    st.compute_persistence()
    pairs = [np.sqrt(st.persistence_intervals_in_dimension(d)) for d in range(3)]
    return pairs

In [ ]:
timers_reg = [timeit.Timer("pers_reg(points)", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_reg = [timer.autorange() for timer in timers_reg]
time_reg = [x[1]/x[0] for x in results_reg]

In [ ]:
timers_reg3d = [timeit.Timer("calc_persistence(points)", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_reg3d = [timer.autorange() for timer in timers_reg3d]
time_reg3d = [x[1]/x[0] for x in results_reg3d]

In [ ]:
timers_per = [timeit.Timer("calc_persistence(points, boxsize=1)", f"points = np.random.rand({N}, 3)", globals=globals()) for N in N_vals]
results_per = [timer.autorange() for timer in timers_per]
time_per = [x[1]/x[0] for x in results_per]

In [ ]:
plt.plot(N_vals, time_reg, 'o', label="regular triangulation")
plt.plot(N_vals, time_reg3d, 'o', label="regular (3D) triangulation")
plt.plot(N_vals, time_per, 'o', label="periodic triangulation")
plt.loglog()
plt.xlabel("$N_{points}$")
plt.ylabel("runtime  [s]")
plt.legend(frameon=True)
plt.show()

In [ ]:
fit_reg = stats.linregress(np.log(N_vals), np.log(time_reg))
fit_per = stats.linregress(np.log(N_vals)[4:], np.log(time_per)[4:])

factor = np.mean(np.array(time_per)[4:] / np.array(time_reg)[4:])
print(f"Periodic triangulation is ~ {factor:.1f} times slower than the regular triangulation (for >500 points)")

# Compare periodic / non-periodic topological summaries

Using random points uniformly distributied inside a box $[0, L)$.

In [ ]:
N = 500
boxsize = 1
lbar = boxsize / np.cbrt(N)
points = boxsize * np.random.rand(N, 3)

In [ ]:
pairs_periodic = calc_persistence(points, boxsize=boxsize)

st = gd.AlphaComplex(points=points).create_simplex_tree()
st.compute_persistence()
pairs_regular = [np.sqrt(st.persistence_intervals_in_dimension(d)) for d in range(3)]

### Persistence diagrams

In [ ]:
plt.figure(figsize=(14,6))
colors = ['b', 'g', 'r']
plt.subplot(1,2,1)
plt.title("periodic triangulation")
for dim in range(3):
    plt.plot(*np.array(pairs_periodic[dim]).T, 'o', alpha=0.6, label=f"$H_{{{dim}}}$")
plt.plot([0, 0.2], [0, 0.2], 'r-')
plt.legend(frameon=True)

plt.subplot(1,2,2)
plt.title("regular triangulation")
for dim in range(3):
    plt.plot(*np.array(pairs_regular[dim]).T, 'o', alpha=0.6, label=f"$H_{{{dim}}}$")
plt.plot([0, 0.2], [0, 0.2], 'r-')
plt.show()

### Weighted silhouttes

In [ ]:
DS = gdr.preprocessing.DiagramSelector(use=True)
SIL = gdr.Silhouette(weight=lambda x: np.power(x[1] - x[0], 1), resolution=500, sample_range=[0, 0.2])
x = np.linspace(*SIL.sample_range, SIL.resolution)

In [ ]:
sils_periodic = SIL.fit_transform(DS.fit_transform([np.array(p) for p in pairs_periodic[1:]]))
sils_regular = SIL.fit_transform(DS.fit_transform([np.array(p) for p in pairs_regular[1:]]))

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.title("$H_1$")
plt.plot(x, sils_periodic[0], label="periodic")
plt.plot(x, sils_regular[0], label="regular")
plt.xlim(0.02, 0.15)
plt.legend(frameon=True)

plt.subplot(1,2,2)
plt.title("$H_2$")
plt.plot(x, sils_periodic[1], label="periodic")
plt.plot(x, sils_regular[1], label="regular")
plt.xlim(0.06, 0.18)
plt.show()

per_peak = x[np.argmax(sils_periodic, axis=1)] / lbar
reg_peak = x[np.argmax(sils_regular, axis=1)] / lbar
print(f"Periodic:  H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")

Individual curves do not match that well.

Next, generate many samples of regular triangulations by shifting center of periodic box

Compare mean to periodic curve

In [ ]:
n_samples = 50
shifted_centers = [utils.center_box_pbc(points, boxsize*np.random.rand(3), boxsize) + boxsize/2 for i in range(n_samples)]

In [ ]:
sts = [gd.AlphaComplex(points).create_simplex_tree() for points in shifted_centers]
for sti in sts: sti.compute_persistence()
pairs_samples = [[np.sqrt(sti.persistence_intervals_in_dimension(d)) for sti in sts] for d in range(3)]

In [ ]:
sils_samples = [SIL.fit_transform(p) for p in pairs_samples[1:]]
sils_reg_mean = [np.mean(s, axis=0) for s in sils_samples]
sils_reg_std = [np.std(s, axis=0) for s in sils_samples]

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.title("$H_1$")
plt.plot(x, sils_periodic[0], label="periodic")
plt.plot(x, sils_reg_mean[0], label="regular")
plt.fill_between(x, sils_reg_mean[0]-sils_reg_std[0], sils_reg_mean[0]+sils_reg_std[0], alpha=0.3, color='g')
plt.xlim(0.02, 0.15)
plt.legend(frameon=True)

plt.subplot(1,2,2)
plt.title("$H_2$")
plt.plot(x, sils_periodic[1], label="periodic")
plt.plot(x, sils_reg_mean[1], label="regular")
plt.fill_between(x, sils_reg_mean[1]-sils_reg_std[1], sils_reg_mean[1]+sils_reg_std[1], alpha=0.3, color='g')
plt.xlim(0.06, 0.18)
plt.show()

per_peak = x[np.argmax(sils_periodic, axis=1)] / lbar
reg_peak = x[np.argmax(sils_reg_mean, axis=1)] / lbar
print(f"Periodic:  H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")

### Betti curves

In [ ]:
BC = gdr.BettiCurve(resolution=SIL.resolution, sample_range=SIL.sample_range)

In [ ]:
bc_periodic = BC.fit_transform(DS.fit_transform([np.array(p) for p in pairs_periodic]))
bc_regular = BC.fit_transform(DS.fit_transform([np.array(p) for p in pairs_regular]))

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, bc_periodic[0], label="periodic")
plt.plot(x, bc_regular[0], label="regular")
plt.xlim(0, 0.12)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, bc_periodic[1], label="periodic")
plt.plot(x, bc_regular[1], label="regular")
plt.xlim(0, 0.18)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, bc_periodic[2], label="periodic")
plt.plot(x, bc_regular[2], label="regular")
plt.xlim(0.05, 0.2)
plt.show()

per_halfmax = x[np.argmin(np.abs(bc_periodic[0] - 0.5*bc_periodic[0][0]))] / lbar
reg_halfmax = x[np.argmin(np.abs(bc_regular[0] - 0.5*bc_regular[0][0]))] / lbar
per_peak = x[np.argmax(bc_periodic[1:], axis=1)] / lbar
reg_peak = x[np.argmax(bc_regular[1:], axis=1)] / lbar
print(f"Periodic:  H_0 halfmax: {per_halfmax:.3f} lbar\t H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_0 halfmax: {reg_halfmax:.3f} lbar\t H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")

Look at full distribution of regular curves

In [ ]:
bc_samples = [BC.fit_transform(DS.fit_transform(p)) for p in pairs_samples]
bc_reg_mean = [np.mean(b, axis=0) for b in bc_samples]
bc_reg_std = [np.std(b, axis=0) for b in bc_samples]

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, bc_periodic[0], label="periodic")
plt.plot(x, bc_reg_mean[0], label="regular")
plt.fill_between(x, bc_reg_mean[0]-bc_reg_std[0], bc_reg_mean[0]+bc_reg_std[0], alpha=0.3, color='g')
plt.xlim(0, 0.12)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, bc_periodic[1], label="periodic")
plt.plot(x, bc_reg_mean[1], label="regular")
plt.fill_between(x, bc_reg_mean[1]-bc_reg_std[1], bc_reg_mean[1]+bc_reg_std[1], alpha=0.3, color='g')
plt.xlim(0, 0.18)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, bc_periodic[2], label="periodic")
plt.plot(x, bc_reg_mean[2], label="regular")
plt.fill_between(x, bc_reg_mean[2]-bc_reg_std[2], bc_reg_mean[2]+bc_reg_std[2], alpha=0.3, color='g')
plt.xlim(0.05, 0.2)
plt.show()

per_halfmax = x[np.argmin(np.abs(bc_periodic[0] - 0.5*bc_periodic[0][0]))] / lbar
reg_halfmax = x[np.argmin(np.abs(bc_reg_mean[0] - 0.5*bc_reg_mean[0][0]))] / lbar
per_peak = x[np.argmax(bc_periodic[1:], axis=1)] / lbar
reg_peak = x[np.argmax(bc_reg_mean[1:], axis=1)] / lbar
print(f"Periodic:  H_0 halfmax: {per_halfmax:.3f} lbar\t H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_0 halfmax: {reg_halfmax:.3f} lbar\t H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")

### Entropy summary curves

cleaner and provide better comparisons than the Betti curves

In [ ]:
ES = gdr.Entropy(mode="vector", normalized=True, resolution=SIL.resolution, sample_range=SIL.sample_range)

In [ ]:
es_periodic = ES.fit_transform(DS.fit_transform([np.array(p) for p in pairs_periodic]))
es_regular = ES.fit_transform(DS.fit_transform([np.array(p) for p in pairs_regular]))

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, es_periodic[0], label="periodic")
plt.plot(x, es_regular[0], label="regular")
plt.xlim(0, 0.12)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, es_periodic[1], label="periodic")
plt.plot(x, es_regular[1], label="regular")
plt.xlim(0, 0.18)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, es_periodic[2], label="periodic")
plt.plot(x, es_regular[2], label="regular")
plt.xlim(0.05, 0.2)
plt.show()

per_halfmax = x[np.argmin(np.abs(es_periodic[0] - 0.5*es_periodic[0][0]))] / lbar
reg_halfmax = x[np.argmin(np.abs(es_regular[0] - 0.5*es_regular[0][0]))] / lbar
per_peak = x[np.argmax(es_periodic[1:], axis=1)] / lbar
reg_peak = x[np.argmax(es_regular[1:], axis=1)] / lbar
print(f"Periodic:  H_0 halfmax: {per_halfmax:.3f} lbar\t H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_0 halfmax: {reg_halfmax:.3f} lbar\t H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")

Look at full distribution of regular curves

In [ ]:
es_samples = [ES.fit_transform(DS.fit_transform(p)) for p in pairs_samples]
es_reg_mean = [np.mean(e, axis=0) for e in es_samples]
es_reg_std = [np.std(e, axis=0) for e in es_samples]

In [ ]:
plt.figure(figsize=(22,6))
plt.subplot(1,3,1)
plt.title("$H_0$")
plt.plot(x, es_periodic[0], label="periodic")
plt.plot(x, es_reg_mean[0], label="regular")
plt.fill_between(x, es_reg_mean[0]-es_reg_std[0], es_reg_mean[0]+es_reg_std[0], alpha=0.3, color='g')
plt.xlim(0, 0.12)
plt.legend(frameon=True)

plt.subplot(1,3,2)
plt.title("$H_1$")
plt.plot(x, es_periodic[1], label="periodic")
plt.plot(x, es_reg_mean[1], label="regular")
plt.fill_between(x, es_reg_mean[1]-es_reg_std[1], es_reg_mean[1]+es_reg_std[1], alpha=0.3, color='g')
plt.xlim(0, 0.18)

plt.subplot(1,3,3)
plt.title("$H_2$")
plt.plot(x, es_periodic[2], label="periodic")
plt.plot(x, es_reg_mean[2], label="regular")
plt.fill_between(x, es_reg_mean[2]-es_reg_std[2], es_reg_mean[2]+es_reg_std[2], alpha=0.3, color='g')
plt.xlim(0.05, 0.2)
plt.show()

per_halfmax = x[np.argmin(np.abs(es_periodic[0] - 0.5*es_periodic[0][0]))] / lbar
reg_halfmax = x[np.argmin(np.abs(es_reg_mean[0] - 0.5*es_reg_mean[0][0]))] / lbar
per_peak = x[np.argmax(es_periodic[1:], axis=1)] / lbar
reg_peak = x[np.argmax(es_reg_mean[1:], axis=1)] / lbar
print(f"Periodic:  H_0 halfmax: {per_halfmax:.3f} lbar\t H_1 peak: {per_peak[0]:.3f} lbar\t H_2 peak: {per_peak[1]:.3f} lbar")
print(f"Regular:   H_0 halfmax: {reg_halfmax:.3f} lbar\t H_1 peak: {reg_peak[0]:.3f} lbar\t H_2 peak: {reg_peak[1]:.3f} lbar")